In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
text[:8]

'First Ci'

In [3]:
len(text)

1115394

In [4]:
chars = sorted(list(set(''.join(text))))
stoi = {c : i for i, c in enumerate(chars)}
itos = {i : c for c, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
vocab_size = len(itos)
print(itos)
print(vocab_size)

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
65


In [5]:
# train and test split
data = torch.tensor(encode(text), dtype=torch.long)
n_train = int(0.9*len(data))

train_data = data[:n_train]       # 90%
eval_data = data[n_train:]         # 10%
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [6]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
epochs = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [7]:
torch.manual_seed(1337)

In [8]:
def get_batch(split):
    # generate small batch of data of inputs data X and targets y
    data = train_data if split == 'train' else eval_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

In [9]:
X, y = get_batch('eval')
X[:2], y[:2]

(tensor([[59, 56, 43, 42,  1, 51, 43,  1, 57, 39, 63,  1, 51, 63,  1, 51, 47, 52,
          42,  6,  0, 13, 52, 42,  1, 47, 44,  1, 63, 53, 59,  1],
         [47, 58,  1, 52, 53, 58,  1, 58, 46, 43, 52,  1, 53, 59, 56,  1, 43, 63,
          43, 50, 47, 42, 57,  1, 57, 47, 52, 49, 12,  1, 21,  1]]),
 tensor([[56, 43, 42,  1, 51, 43,  1, 57, 39, 63,  1, 51, 63,  1, 51, 47, 52, 42,
           6,  0, 13, 52, 42,  1, 47, 44,  1, 63, 53, 59,  1, 41],
         [58,  1, 52, 53, 58,  1, 58, 46, 43, 52,  1, 53, 59, 56,  1, 43, 63, 43,
          50, 47, 42, 57,  1, 57, 47, 52, 49, 12,  1, 21,  1, 44]]))

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'eval']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, y = get_batch(split)
            logits, loss = model(X, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class AttentionHead(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, X):
        # input size: (batch, time-step, channels) == (batch_size, block_size, n_embd)
        # output size: (batch, time-step, head size) == (batch_size, block_size, head_size)
        B, T, C = X.shape
        k = self.key(X)   # (B, T, hs)
        q = self.query(X) # (B, T, hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * k.shape[-1] **-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(X) # (B, T, hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out
        

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-atttention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, X):
        out = torch.cat([h(X) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linerity"""
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # in transformer paper, the dimension is 512 and projectino to 2048, so it's 4 times
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, X):
        return self.net(X)


class Block(nn.Module):
    """ Transformer block: commnunication followed by computation"""
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, X):
        X = X + self.sa(self.ln1(X))
        X = X + self.ffwd(self.ln2(X))
        return X


class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)    # (vocab_size, C)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # (T, C)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        # better init, important
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, X, y=None):
        # X and y are both (B, T) tensor integers, B = batch_size, T = block_size
        B, T = X.shape
        
        tok_emb = self.token_embedding_table(X) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        X = tok_emb + pos_emb
        X = self.blocks(X) # (B, T, C)
        X = self.ln_f(X)   # (B, T, C)
        logits = self.lm_head(X) # (B, T, vocab_size)
        
        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = F.cross_entropy(logits, y)
        return logits, loss
    
    def generate(self, X, max_new_tokens):
        # X is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop X to the last block_size tokens
            X_cond = X[:, -block_size:]
            # get the predictions
            logits, loss = self(X_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_nxt = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            X = torch.cat((X, idx_nxt), dim=1) # (B, T+1)
        return X
        

In [12]:
model = GPTLanguageModel(vocab_size)
model = model.to(device)

In [13]:
print('parameters number:', sum(p.nelement() for p in model.parameters()))

parameters number: 209729


In [14]:
# train model
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(epochs):
    # every once in a while evaluate the loss on train and eval sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, eval loss {losses['eval']:.4f}")
    
    # sample a batch of data
    Xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(Xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.1405, eval loss 4.1400
step 100: train loss 2.6153, eval loss 2.6084
step 200: train loss 2.4567, eval loss 2.4561
step 300: train loss 2.3705, eval loss 2.3839
step 400: train loss 2.3026, eval loss 2.3068
step 500: train loss 2.2183, eval loss 2.2404
step 600: train loss 2.1790, eval loss 2.2249
step 700: train loss 2.1254, eval loss 2.1574
step 800: train loss 2.0916, eval loss 2.1378
step 900: train loss 2.0478, eval loss 2.1056
step 1000: train loss 2.0408, eval loss 2.0891
step 1100: train loss 1.9999, eval loss 2.0652
step 1200: train loss 1.9740, eval loss 2.0666
step 1300: train loss 1.9613, eval loss 2.0449
step 1400: train loss 1.9421, eval loss 2.0409
step 1500: train loss 1.9028, eval loss 2.0020
step 1600: train loss 1.8864, eval loss 1.9970
step 1700: train loss 1.8835, eval loss 1.9921
step 1800: train loss 1.8862, eval loss 2.0023
step 1900: train loss 1.8473, eval loss 1.9790
step 2000: train loss 1.8518, eval loss 1.9652
step 2100: train loss 1.8

In [17]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))


May him I vows their dispice-with sedwain your and
Not mid loruniven, awas yet a nixite,
Ay aid, to whath; me seach mad this me?
Slay you have onten sent.

KI ING HENRY G ROK:
What's that a whed no migh Ad Justries post:
O ducceday you hold shall be country in the dire of Payullain:
Nay, you suir ad; now, bole
Shall that then youm crie, and lass; lady? 'I would must not much oar.

HISCWISTAnges Nambling of sutingly you, whethere intemond heaven?

PETOLYCUS:
Nay, and thee kill
Who we prering sepeak then for of airnot
Hath, Unnce, 'fulty is?

First Must SERMIOW:
I abaginage, I by must that and the
stooke fainst gives to that.

VISTIS:
For, if you a pure that shamAes.
Oxted oflowerly I land him this shall seaus; say I came postigervet
That the moticke to run has offerit,
This than this it moore; shall heart again' deadfore
To Richanting to consta, and yas
And I have ownd faulty of ouch no Edways reasong:
This themself, to of hear. Wolloow yong; where tell footh?


MARCIUS:
thouse my pari